In [104]:
#loading and preparing data
from datasets import load_dataset,Dataset
import pandas as pd

import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"




In [105]:
base_model='xlm-roberta-base'

In [106]:
dataset = load_dataset("csv", data_files="../Data/Cleaned/review.csv")

# train-test split
dataset = dataset["train"].train_test_split(test_size=0.2, seed=42)

train_dataset = dataset["train"]
test_dataset  = dataset["test"]

#drop none values
train_dataset = train_dataset.filter(lambda example: example["review"] is not None)
test_dataset = test_dataset.filter(lambda example: example["review"] is not None)


In [107]:
dataset["test"]

Dataset({
    features: ['review', 'rating'],
    num_rows: 306
})

In [108]:
from transformers import (
    XLMRobertaTokenizerFast,
    XLMRobertaForSequenceClassification,
    TrainingArguments,
    Trainer
)

model="xlm-roberta-base"


tokenizer = XLMRobertaTokenizerFast.from_pretrained(
    model
)



In [109]:
def tokenize(text):
    return tokenizer(text['review'],padding='max_length',truncation=True,max_length=128)

In [110]:
def preprocess(ds):
    ds = ds.map(tokenize, remove_columns=["review"])  # remove raw text (saves memory)
    ds = ds.rename_column("rating", "labels")
    ds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
    return ds


train_dataset = preprocess(train_dataset)
test_dataset  = preprocess(test_dataset)



In [111]:
model=XLMRobertaForSequenceClassification.from_pretrained(model,num_labels=3)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [112]:
#show all layers of model
for layer in model.base_model.encoder.layer:
    print(layer)



XLMRobertaLayer(
  (attention): XLMRobertaAttention(
    (self): XLMRobertaSdpaSelfAttention(
      (query): Linear(in_features=768, out_features=768, bias=True)
      (key): Linear(in_features=768, out_features=768, bias=True)
      (value): Linear(in_features=768, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (output): XLMRobertaSelfOutput(
      (dense): Linear(in_features=768, out_features=768, bias=True)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (intermediate): XLMRobertaIntermediate(
    (dense): Linear(in_features=768, out_features=3072, bias=True)
    (intermediate_act_fn): GELUActivation()
  )
  (output): XLMRobertaOutput(
    (dense): Linear(in_features=3072, out_features=768, bias=True)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
)
XLMRobertaLayer(
  (attention): XLMRobert

In [113]:
from transformers import TrainingArguments
model.cpu()

training_args = TrainingArguments(
    output_dir="./Models/xlm-finetuned-sentiment",
    num_train_epochs=1,
    per_device_train_batch_size=8,
    logging_dir="./logs",
    learning_rate=2e-5,
    weight_decay=0.01,
    report_to="none",
    no_cuda=True
)



In [114]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)


AcceleratorError: CUDA error: device-side assert triggered
Search for `cudaErrorAssert' in https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__TYPES.html for more information.
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
trainer.train()


AcceleratorError: CUDA error: device-side assert triggered
Search for `cudaErrorAssert' in https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__TYPES.html for more information.
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
